In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [2]:
app = Flask(__name__)
received_data = []

In [3]:
def your_meeting_assistant(data): 
    # Your Agentic AI Calls 
    data["EventStart"] = ""
    data["EventEnd"] = ""
    data["Duration_mins"] = ""
    return data

In [4]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [5]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.177.1:5000
Press CTRL+C to quit
106.194.221.202 - - [12/Jul/2025 11:02:08] "GET / HTTP/1.1" 404 -
106.194.221.202 - - [12/Jul/2025 11:02:09] "GET /favicon.ico HTTP/1.1" 404 -


In [9]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from datetime import datetime, timedelta

# Utility to pull Google Calendar busy slots
def get_busy_slots_for_attendee(creds, email, meeting_date):
    service = build('calendar', 'v3', credentials=creds)
    start_of_day = meeting_date.replace(hour=0, minute=0, second=0, microsecond=0).isoformat() + 'Z'
    end_of_day = meeting_date.replace(hour=23, minute=59, second=59, microsecond=0).isoformat() + 'Z'
    
    events_result = service.events().list(
        calendarId=email,
        timeMin=start_of_day,
        timeMax=end_of_day,
        singleEvents=True,
        orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    busy_slots = []
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        end = event['end'].get('dateTime', event['end'].get('date'))
        busy_slots.append((
            datetime.fromisoformat(start),
            datetime.fromisoformat(end)
        ))
    return busy_slots

# Slot finding algorithm
def find_earliest_free_slot(busy_slots_list, meeting_date, meeting_duration_mins):
    work_start = meeting_date.replace(hour=9, minute=0, second=0, microsecond=0)
    work_end = meeting_date.replace(hour=18, minute=0, second=0, microsecond=0)
    slot_duration = timedelta(minutes=meeting_duration_mins)

    current = work_start
    while current + slot_duration <= work_end:
        conflict = False
        for busy_slots in busy_slots_list:
            for busy_start, busy_end in busy_slots:
                if current < busy_end and (current + slot_duration) > busy_start:
                    conflict = True
                    break
            if conflict:
                break
        if not conflict:
            return current, current + slot_duration
        current += timedelta(minutes=15)
    return None, None  # No available slot found


In [11]:
from datetime import datetime, timedelta
import json
from openai import OpenAI
from google.oauth2.credentials import Credentials

client = OpenAI(
    base_url="http://127.0.0.1:3000/v1",
    api_key="EMPTY"
)

def your_meeting_assistant(request_json):
    # 📌 Mapping emails to their respective token files
    email_to_tokenfile = {
        "userone.amd@gmail.com": "/home/user/AI-Scheduling-Assistant/user1_token.json",
        "usertwo.amd@gmail.com": "/home/user/AI-Scheduling-Assistant/user2_token.json",
        "userthree.amd@gmail.com": "/home/user/AI-Scheduling-Assistant/user3_token.json",
    }

    # 1️⃣ Extract EmailContent
    email_content = request_json["EmailContent"]

    # 2️⃣ Use DeepSeek to parse participants, duration, time constraints
    prompt = f"""
You are a structured extraction agent. Given the email content below, extract:
- participants (list of emails, if only names then append '@amd.com')
- meeting_duration (in minutes, integer)
- time_constraints (like "Thursday", "next week", etc.)

Return STRICTLY JSON with:
{{
    "participants": [...],
    "meeting_duration": ...,
    "time_constraints": "..."
}}

EmailContent:
\"\"\"
{email_content}
\"\"\"
"""
    response = client.chat.completions.create(
        model="/home/user/Models/deepseek-ai/deepseek-llm-7b-chat",
        messages=[{"role": "user", "content": prompt}]
    )
    parsed = json.loads(response.choices[0].message.content)

    # 3️⃣ Resolve "Thursday" to actual date
    base_datetime = datetime.strptime(request_json["Datetime"], "%d-%m-%YT%H:%M:%S")
    weekdays = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
        "friday": 4, "saturday": 5, "sunday": 6
    }
    tc = parsed["time_constraints"].strip().lower()
    meeting_date = base_datetime
    for day in weekdays:
        if day in tc:
            days_ahead = (weekdays[day] - base_datetime.weekday()) % 7
            if days_ahead == 0:
                days_ahead = 7
            meeting_date = base_datetime + timedelta(days=days_ahead)
            break

    # 4️⃣ Collect busy slots for all attendees using **their specific tokens**
    attendee_emails = [request_json["From"]] + [a["email"] for a in request_json["Attendees"]]
    busy_slots_list = []

    for email in attendee_emails:
        try:
            token_file = email_to_tokenfile[email]
            creds = Credentials.from_authorized_user_file(
                token_file, ['https://www.googleapis.com/auth/calendar.readonly']
            )
            busy_slots = get_busy_slots_for_attendee(creds, email, meeting_date)
            busy_slots_list.append(busy_slots)
        except Exception as e:
            print(f"Error fetching calendar for {email}: {e}")

    # 5️⃣ Find the earliest free slot
    start_time, end_time = find_earliest_free_slot(
        busy_slots_list, meeting_date, int(parsed["meeting_duration"])
    )
    if not start_time:
        raise Exception("No available slot found for meeting.")

    # 6️⃣ Build 'processed' JSON
    processed_json = request_json.copy()
    processed_json["Start"] = start_time.isoformat()
    processed_json["End"] = end_time.isoformat()
    processed_json["Duration_mins"] = str(parsed["meeting_duration"])

    # 7️⃣ Build 'output' JSON
    events = []
    for email in attendee_emails:
        events.append({
            "email": email,
            "events": [
                {
                    "StartTime": start_time.isoformat(),
                    "EndTime": end_time.isoformat(),
                    "NumAttendees": len(attendee_emails),
                    "Attendees": attendee_emails,
                    "Summary": request_json["Subject"]
                }
            ]
        })

    output_json = request_json.copy()
    output_json["Attendees"] = events
    output_json["EventStart"] = start_time.isoformat()
    output_json["EventEnd"] = end_time.isoformat()
    output_json["Duration_mins"] = str(parsed["meeting_duration"])

    return {
        "processed": processed_json,
        "output": output_json
    }


In [8]:
with open("/home/user/AI-Scheduling-Assistant/1_Input_Request.json") as f:
    req = json.load(f)

result = your_meeting_assistant(req)
print(json.dumps(result, indent=2))


{
  "processed": {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
      {
        "email": "usertwo.amd@gmail.com"
      },
      {
        "email": "userthree.amd@gmail.com"
      }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
    "Start": "2025-07-10T10:30:00",
    "End": "2025-07-10T11:00:00",
    "Duration_mins": "30"
  },
  "output": {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
      {
        "email": "userone.amd@gmail.com",
        "events": [
          {
            "StartTime": "2025-07-10T10:30:00",
            "EndTime": "2025-07-10T11:00:00",
            "NumAttendees": 3,
   

In [12]:
with open("/home/user/AI-Scheduling-Assistant/1_Input_Request.json") as f:
    req = json.load(f)

result = your_meeting_assistant(req)
print(json.dumps(result, indent=2))

Error fetching calendar for userone.amd@gmail.com: [Errno 2] No such file or directory: '/home/user/AI-Scheduling-Assistant/user1_token.json'
Error fetching calendar for usertwo.amd@gmail.com: [Errno 2] No such file or directory: '/home/user/AI-Scheduling-Assistant/user2_token.json'
Error fetching calendar for userthree.amd@gmail.com: [Errno 2] No such file or directory: '/home/user/AI-Scheduling-Assistant/user3_token.json'
{
  "processed": {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "09-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
      {
        "email": "usertwo.amd@gmail.com"
      },
      {
        "email": "userthree.amd@gmail.com"
      }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project.",
    "Start": "2025-07-10T09:00:00",
    "End": "2025-07-10T09:30:00",
    "Durat